In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

** Retreiving the zipcode data from previous pickled file**  

In [ ]:
zipu=pd.read_pickle("zip_code_unique.pkl")
zipu=zipu[0].unique()

** Scraping the data from www.city-data.com **  
Male popluation , female population, and median household income is extracted

In [146]:
responses=[]
male=[]
female=[]
income_list=[]
for z in zipu:
    #print("http://www.city-data.com/zips/"+z)
    #print(z)
    response=requests.get("http://www.city-data.com/zips/"+z+".html")
    if(response.status_code==200):
        
        soup = BeautifulSoup(response.text, "html.parser")
        income = soup.find("b", text=re.compile("^Estimated median household income in 2016:"))
        male_population = soup.find("b", text=re.compile("^Males:"))
        female_population = soup.find("b", text=re.compile("^Females:"))
        
        male.append(male_population.next_sibling)
        female.append(female_population.next_sibling)
        income_list.append(income.next_sibling.next_element.text)
        
    else:
        #print("oops")
        male.append(",")
        female.append(",")
        income_list.append("oops$")
t=list(zip(male,female,income))        

In [147]:
inc=[i.split("$")[1].replace(',',"") for i in income_list]
male_population=[i.strip().replace(',',"") for i in male]
female_population=[i.strip().replace(',',"") for i in female]


In [148]:
relational_zipcode_df=pd.DataFrame(list(zip(zipu,male_population,female_population,inc)),columns=["Zipcode","Male_population","Female_population","Median_income_zipcode"])

In [149]:
pd.to_pickle(relational_zipcode_df,"relational_zipcode_df.pkl")